In [6]:
import numpy as np
import optuna
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from hftbacktest import Recorder
from hftbacktest.stats import LinearAssetRecord
from strategies.glft import gridtrading_glft_mm

# Data Input paths
preprocessed_data_path = "../data/daily_processed"
daily_eod_snapshots = "../data/snapshots"

In [7]:
# Taken from https://support.deribit.com/hc/en-us/articles/25944746248989-Fees
MAKER_FEE = -0.0001   
TAKER_FEE =  0.0005    

max_position=50

In [8]:
day_start = 2
day_end = 22
n_trading_days = day_end - day_start

def objective(trial):
    data_paths = []
    for i in range(day_start, day_end):
        day_file = f"../data/daily_processed/deribit_eth_perp_2025-01-{i:02d}.npz"
        data_paths.append(day_file)
        
    eod = np.load(f"../data/snapshots/deribit_eth_perp_2025-01-{day_start:02d}_eod.npz")['data']

    # Hyperparameters under optimization
    gamma = trial.suggest_float("gamma", 0.01, 0.1)
    delta = trial.suggest_int("delta", 1, 10)
    adj1 = trial.suggest_float("adj1", 0.01, 2)
    adj2 = trial.suggest_float("adj2", 0.01, 1)

    asset = (
        BacktestAsset()
            .data(data_paths)
            .initial_snapshot(eod)
            .linear_asset(1.0)
            .constant_latency(10000, 10000) 
            .risk_adverse_queue_model()
            .no_partial_fill_exchange()
            .trading_value_fee_model(MAKER_FEE, TAKER_FEE)
            .tick_size(0.01)
            .lot_size(1)
            .last_trades_capacity(10000))
    
    hbt = HashMapMarketDepthBacktest([asset])
    recorder = Recorder(1, n_trading_days* 1_000_000)
    
    gridtrading_glft_mm(hbt, recorder.recorder, n_trading_days, gamma, delta, adj1, adj2, max_position)
    hbt.close()

    stats = LinearAssetRecord(recorder.get(0)).stats()
    return stats.summary(pretty=True)['Return'][0]

In [9]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2025-05-29 13:17:32,109] A new study created in memory with name: no-name-55981e29-bbfe-4052-b186-7cf82990e46d


Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20
Hour: 21
Hour: 22
Hour: 23
Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19


[I 2025-05-29 13:17:51,856] Trial 0 finished with value: 300.0500000000002 and parameters: {'gamma': 0.04370861069626263, 'delta': 10, 'adj1': 1.4666679442046962, 'adj2': 0.6026718993550663}. Best is trial 0 with value: 300.0500000000002.


Hour: 20
Hour: 21
Hour: 22
Hour: 23


In [10]:
print("BEST Parameter Combinations", study.best_params)
print("BEST Return Achieved", study.best_value)

{'gamma': 0.04370861069626263,
 'delta': 10,
 'adj1': 1.4666679442046962,
 'adj2': 0.6026718993550663}